In [23]:
# Импорт библиотек
from Def import *

In [24]:
symbol = "BTCUSD"
interval = "5"
days = 15
df = get_historical_data(symbol, interval, days)

#Создаём столбцы с ADX
df['ADX'],df['DI+'],df['DI-'] = adx(df['high'], df['low'], df['close'], n=14)
macd_line, signal_line, histogram = macd(df)
df['macd_line'] = macd_line
df['signal_line'] = signal_line
df['histogram'] = histogram
df.dropna(inplace=True)

In [25]:
best_profit = 0  # Лучший профит
best_parameters = {}  # Лучшие параметры

# Перебираем параметры ADX и MACD
for adx_threshold in range(20, 40):
    for macd_threshold in range(-2, 3):
        # Создаем столбец "signal" и заполняем его значениями по умолчанию
        df['signal'] = 'hold'
        position = 'out'
        initial_cash = 1000
        initial_cash2 = initial_cash
        initial_coins = 0
        initial_coins = 0
        commission = 0.0001

        # Проходим по строкам таблицы
        for index, row in df.iterrows():
            # Если нет позиции на покупку, но есть условия для покупки
            if position == 'out' and row['ADX'] > adx_threshold and row['macd_line'] > macd_threshold:
                # Устанавливаем сигнал на покупку
                df.at[index, 'signal'] = 'buy'
                coins_bought = (initial_cash * (1 - commission)) / row['close']
                initial_coins += coins_bought
                initial_cash = 0
                position = 'long'

            # Если есть позиция на покупку и есть условие для продажи
            elif position == 'long' and row['ADX'] < adx_threshold and row['macd_line'] < macd_threshold:
                # Устанавливаем сигнал на продажу
                df.at[index, 'signal'] = 'sell'
                coins_sold = initial_coins
                initial_cash += coins_sold * row['close'] * (1 - commission)
                initial_coins = 0
                position = 'out'

        # Если остались криптовалюты в конце, продаем их по последней известной цене
        if position == 'long':
            coins_sold = initial_coins
            initial_cash += coins_sold * df.iloc[-1]['close'] * (1 - commission)
            initial_coins = 0
            position = 'out'

        # Вычисляем профит
        profit = initial_cash - 1000

        # Проверяем, является ли текущий профит лучшим
        if profit > best_profit:
            best_profit = profit
            best_parameters = {'adx_threshold': adx_threshold, 'macd_threshold': macd_threshold}

# Выводим лучшие параметры и профит
print("Best Parameters:", best_parameters)
print("Best Profit: $", best_profit)

Best Parameters: {}
Best Profit: $ 0


In [ ]:
best_profit = 0  # Лучший профит
best_parameters = {}  # Лучшие параметры

# Перебираем параметры ADX, DI+ и DI-, а также MACD
for adx_threshold in range(20, 40):
    for di_plus_threshold in range(20, 40):
        for di_minus_threshold in range(20, 40):
            for macd_line_threshold in range(-2, 3):
                for signal_line_threshold in range(-2, 3):
                    for histogram_threshold in range(-2, 3):
                        # Создаем столбец "signal" и заполняем его значениями по умолчанию
                        df['signal'] = 'hold'
                        position = 'out'
                        initial_cash = 1000
                        initial_coins = 0
                        commission = 0.0001

                        # Проходим по строкам таблицы
                        for index, row in df.iterrows():
                            # Если нет позиции на покупку, но есть условия для покупки
                            if (
                                position == 'out'
                                and row['ADX'] > adx_threshold
                                and row['DI+'] > di_plus_threshold
                                and row['DI-'] > di_minus_threshold
                                and row['macd_line'] > macd_line_threshold
                                and row['signal_line'] > signal_line_threshold
                                and row['histogram'] > histogram_threshold
                            ):
                                # Устанавливаем сигнал на покупку
                                df.at[index, 'signal'] = 'buy'
                                coins_bought = (initial_cash * (1 - commission)) / row['close']
                                initial_coins += coins_bought
                                initial_cash = 0
                                position = 'long'

                            # Если есть позиция на покупку и есть условие для продажи
                            elif (
                                position == 'long'
                                and (
                                    row['ADX'] < adx_threshold
                                    or row['DI+'] < di_plus_threshold
                                    or row['DI-'] < di_minus_threshold
                                    or row['macd_line'] < macd_line_threshold
                                    or row['signal_line'] < signal_line_threshold
                                    or row['histogram'] < histogram_threshold
                                )
                            ):
                                # Устанавливаем сигнал на продажу
                                df.at[index, 'signal'] = 'sell'
                                coins_sold = initial_coins
                                initial_cash += coins_sold * row['close'] * (1 - commission)
                                initial_coins = 0
                                position = 'out'

                        # Если остались криптовалюты в конце, продаем их по последней известной цене
                        if position == 'long':
                            coins_sold = initial_coins
                            initial_cash += coins_sold * df.iloc[-1]['close'] * (1 - commission)
                            initial_coins = 0
                            position = 'out'

                        # Вычисляем профит
                        profit = initial_cash - 1000

                        # Проверяем, является ли текущий профит лучшим
                        if profit > best_profit:
                            best_profit = profit
                            best_parameters = {
                                'adx_threshold': adx_threshold,
                                'di_plus_threshold': di_plus_threshold,
                                'di_minus_threshold': di_minus_threshold,
                                'macd_line_threshold': macd_line_threshold,
                                'signal_line_threshold': signal_line_threshold,
                                'histogram_threshold': histogram_threshold
                            }

# Выводим лучшие параметры и профит
print("Best Parameters:", best_parameters)
print("Best Profit: $", best_profit)
